# Задание 1 (10 баллов)

Обучите языковую модель с помощью GRPO на вот этом датасете - `notbadai/python_functions_reasoning`
Он чем-то похож на датасет из семинара, но вместо математических задач нужно написать код на питоне.
Измените system prompt под эту задачу. Напишите аналогичные reward функции - одна на проверку формата (markdown wrappers ```python... ``` вокруг кода в конце), и на проверку что код работает (можете использовать import ast; ast.parse(string) для проверки)
Если модель сначала не может отвечать в нужно формате, то сделайте цикл SFT обучения на небольшом куске датасета.
GRPO в колабе может работать долго, поэтому не старайтесь пропустить весь датасет. Но обучайте хотя бы на 100 промптах (по несколько генераций на каждый промпт).
Оцените разницу в качестве на небольшом сабсете, прогнав изначальную модель, модель после SFT (если она есть) и GRPO модель. В качестве метрики используйте reward функции.
Можете взять любую языковую модель любого размера. Можете дообучать всю модель или же только адаптера поверх квантизированной модели.


In [ ]:
!pip install trl -q
!pip install -U bitsandbytes -q

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from trl import GRPOConfig, GRPOTrainer, SFTConfig, SFTTrainer
from datasets import load_dataset
from tqdm.notebook import tqdm
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
import re
import numpy as np
import ast

pd.set_option('display.max_rows', 3000)
pd.set_option('display.max_colwidth', 5000)

In [2]:
model_name = "Qwen/Qwen2.5-0.5B-Instruct"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
dataset = load_dataset('notbadai/python_functions_reasoning')

In [5]:
(dataset['train'][0])

{'prompt': '```python\ndef scientific_to_float(scientific_notation):\n    """Create a script to convert a number in scientific notation into a float, considering the following conditions:\n- The script should handle both positive and negative exponents.\n- The script should handle scientific notation with multiple digits before the "e" symbol.\n- The script should handle scientific notation with a decimal point before the "e" symbol.\n- The script should handle scientific notation with multiple digits after the decimal point.\n- The script should handle scientific notation with a negative exponent that requires more than one digit.\n- The script should handle scientific notation with a decimal point after the "e" symbol.\n- The script should handle scientific notation with multiple digits after the decimal point and a negative exponent.\n- The script should handle scientific notation with multiple digits after the decimal point and a positive exponent.\n- The script should handle scien

In [6]:
system_prompt = """
You task is to write Python code. Before outputting the result, give step-by-step reasoning.
Put the final asnwer at the end wrapped in Python markdown preceeding by four hash signes like this:
####
```python
*answer*
```
"""

In [7]:
def chatml_format(example, system_prompt=system_prompt):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": example['prompt']}
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return {"prompt": prompt}

def chatml_format_sft(example, system_prompt=system_prompt):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": example['prompt']},
        {"role": "assistant", "content": f"{example['reasoning']}\n{example['answer']}"}
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return {"text": prompt}

In [8]:
dataset_train = dataset['train'].select([i for i in range(100)])
dataset_test = dataset['train'].select([i for i in range(100, 200)])
dataset_sft = dataset['train'].select([i for i in range(200, 1000)])

In [9]:
original_columns = dataset_train.column_names

dataset_train = dataset_train.map(
    chatml_format,
    remove_columns=original_columns
)
dataset_test = dataset_test.map(
    chatml_format,
    remove_columns=original_columns
)
dataset_sft = dataset_sft.map(
    chatml_format_sft,
    remove_columns=original_columns
)

In [10]:
def format_accuracy_func(completions, **kwargs):
    rewards = []
    for response in completions:
        if re.search('```python.*```', response, re.DOTALL):
            rewards.append(1.0)

        else:
            rewards.append(0.0)

    return rewards

def answer_accuracy_func(completions, **kwargs):
    rewards = []
    for response in completions:
        match_c = re.search('```python(.*?)```', response, re.DOTALL)
        if match_c is not None:
            try:
                ast.parse(match_c.group(1).strip())
                rewards.append(1.0)
            except:
                rewards.append(0.0)
        else:
            rewards.append(0.0)
    return rewards

In [12]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    load_in_4bit=True
)
model.config.use_cache = False

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:
# без дообучения
response = []
dataset_test.set_format("pandas")
batch_size = 2
for i in tqdm(range(0, 100, batch_size)):
    batch = dataset_test[i:i+batch_size]
    input_ids = tokenizer.batch_encode_plus(batch['prompt'].tolist(), return_tensors='pt', padding=True)

    output = model.generate(
        input_ids['input_ids'].to('cuda'), attention_mask=input_ids['attention_mask'].to('cuda'),
        max_new_tokens=1024, do_sample=True, pad_token_id=tokenizer.eos_token_id
    )[:,input_ids['input_ids'].shape[-1]:]
    response.extend(tokenizer.batch_decode(output, skip_special_tokens=True))

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
np.mean(format_accuracy_func(response))

np.float64(0.95)

In [ ]:
np.mean(answer_accuracy_func(response))

np.float64(0.0)

In [13]:
# sft
peft_config = LoraConfig(
    r=32,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules="all-linear"
)

In [14]:
training_args = SFTConfig(
    max_length=1024,
    label_names=["labels"],
    num_train_epochs=3,
    report_to="none",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    logging_steps=10,
)

In [15]:
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=dataset_sft,
    args=training_args,
    peft_config=peft_config
)

Converting train dataset to ChatML:   0%|          | 0/800 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/800 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/800 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/800 [00:00<?, ? examples/s]

In [16]:
trainer.train()

Step,Training Loss
10,1.506900
20,1.149400
30,0.873900
40,0.786000
50,0.651000
60,0.657700
70,0.668300
80,0.607100
90,0.626900
100,0.614300


TrainOutput(global_step=600, training_loss=0.5903805128733317, metrics={'train_runtime': 1168.3863, 'train_samples_per_second': 2.054, 'train_steps_per_second': 0.514, 'total_flos': 2757057307004160.0, 'train_loss': 0.5903805128733317})

In [17]:
trainer.save_model("./Qwen-sft")

In [ ]:
response = []
dataset_test.set_format("pandas")
batch_size = 2
for i in tqdm(range(0, 100, batch_size)):
    batch = dataset_test[i:i+batch_size]
    input_ids = tokenizer.batch_encode_plus(batch['prompt'].tolist(), return_tensors='pt', padding=True)

    output = model.generate(
        input_ids['input_ids'].to('cuda'), attention_mask=input_ids['attention_mask'].to('cuda'),
        max_new_tokens=1024, do_sample=True, pad_token_id=tokenizer.eos_token_id
    )[:,input_ids['input_ids'].shape[-1]:]
    response.extend(tokenizer.batch_decode(output, skip_special_tokens=True))

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
np.mean(format_accuracy_func(response))

np.float64(0.99)

In [ ]:
np.mean(answer_accuracy_func(response))

np.float64(0.66)

In [11]:
# grpo
model_name = "./Qwen-sft"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    load_in_4bit=True
).to('cuda')
model.config.use_cache = False

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [12]:
peft_config = LoraConfig(
    r=32,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

In [13]:
training_args = GRPOConfig(logging_steps=10,
                           report_to="none",
                           num_generations=8,
                           num_train_epochs=1,
                           temperature=1.5,
                           label_names=["labels"])

In [14]:
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[answer_accuracy_func, format_accuracy_func],
    args=training_args,
    train_dataset=dataset_train,
    peft_config=peft_config
)

In [15]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss
10,0.039700
20,0.114100
30,-0.005800
40,0.084800
50,-0.010000
60,0.099900
70,0.061600
80,-0.009300
90,0.064600
100,0.007300


TrainOutput(global_step=100, training_loss=0.0446816498786211, metrics={'train_runtime': 2995.1003, 'train_samples_per_second': 0.033, 'train_steps_per_second': 0.033, 'total_flos': 0.0, 'train_loss': 0.0446816498786211})

In [16]:
trainer.save_model("./Qwen-grpo")

In [17]:
response = []
dataset_test.set_format("pandas")
batch_size = 2
for i in tqdm(range(0, 100, batch_size)):
    batch = dataset_test[i:i+batch_size]
    input_ids = tokenizer.batch_encode_plus(batch['prompt'].tolist(), return_tensors='pt', padding=True)

    output = model.generate(
        input_ids['input_ids'].to('cuda'), attention_mask=input_ids['attention_mask'].to('cuda'),
        max_new_tokens=1024, do_sample=True, pad_token_id=tokenizer.eos_token_id
    )[:,input_ids['input_ids'].shape[-1]:]
    response.extend(tokenizer.batch_decode(output, skip_special_tokens=True))

  0%|          | 0/50 [00:00<?, ?it/s]

In [18]:
np.mean(format_accuracy_func(response))

np.float64(0.95)

In [19]:
np.mean(answer_accuracy_func(response))

np.float64(0.83)